# WhiteboxTools no Python: Hidrologia

### Prof. Carlos H. Grohmann - IEE-USP  

### Exemplo de uso do WhiteboxTools com análise hidrológica de MDE  

adaptado de: https://jblindsay.github.io/WhiteboxTutorials/WbW_tutorials/WbW_tutorial1.html

veja também: https://github.com/opengeos/whitebox-python

## Instalação WBT e setup

In [2]:
# Lembre-se que você precisa ter o WhiteboxTools instalado. Se não tiver, pode fazer o download na próxima célula
!pip install whitebox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 3.4 MB/s eta 0:00:00


In [1]:
# setup inicial - importe o módulo e defina a localização do executavel (ou faça o download)
import whitebox
# whitebox.download_wbt(linux_musl=True, reset=True)
wbt = whitebox.WhiteboxTools()

In [2]:
wbt.set_whitebox_dir('/mnt/c/Users/PC/Documents/py3/lib/python3.8/site-packages/whitebox/WBT')
wbt.set_working_dir('/mnt/d/dados_SPAMLab/amaz_hidro')

# wbt.set_whitebox_dir('/home/guano/Documents/mcenv/lib/python3.11/site-packages/whitebox')
# wbt.set_working_dir('/home/guano/Dropbox/USP/extensao/videos_whitebox')

# Setting the following to True enables tools to output DEFLATE compressed GeoTIFFs.
# You only need to do this once, if you wish all tools to compress their raster
# outputs.
wbt.set_compress_rasters(True) 

0

In [13]:
import math as m

## Trabalhando com CopDEM para a bacia Amazônica

In [3]:
tif = 'copdem_15sec_amaz_albers.tif'
tname = tif[:-4] # nome do arquivo sem extensão '.tif'   

In [4]:
# relevo sombreado
az = 45
alt = 35
zf = 10
shade = f'{tname}_shade_{az}_{alt}_z{zf}'

wbt.hillshade(
dem = tif,
output = shade,
azimuth=az,
altitude=alt,
zfactor=zf)

./whitebox_tools --run="Hillshade" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --dem='copdem_15sec_amaz_albers.tif' --output='copdem_15sec_amaz_albers_shade_45_35_z10' --azimuth=45 --altitude=35 --zfactor='10' -v --compress_rasters=True

****************************
* Welcome to Hillshade     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Performing analysis: 0%
Performing analysis: 1%
Performing analysis: 2%
Performing analysis: 3%
Performing analysis: 4%
Performing analysis: 5%
Performing analysis: 6%
Performing analysis: 7%
Performing analysis: 8%
Performing analysis: 9%
Performing analysis: 10%
Performing analysis: 11%
Performing analysis: 12%
Performing analysis: 13%
Performing analysis: 14%
Performing analysis: 15%
Performing analysis: 16%
Performing analysis: 17%
Performing analysis: 18%
Performing analysis: 19%
Performing analysis: 20%
Performing analysis: 21%
Performing analysis: 22%
Performing analysis: 23%
Performing analy

0

In [5]:
# suavizar MDE pela media
# w = 5
# mean = f'{tname}_mean_{w}x{w}'
# wbt.mean_filter(
#     i = tif, 
#     output = mean+'.tif', 
#     filterx=w, 
#     filtery=w)

In [6]:
# breach depressions
dist = 3000
breach = f'{tname}_breached_d{dist}'
wbt.breach_depressions_least_cost(
    dem = tif, 
    output = breach, 
    dist = dist, 
    max_cost=None, 
    min_dist=True, 
    flat_increment=None, 
    fill=True)

./whitebox_tools --run="BreachDepressionsLeastCost" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --dem='copdem_15sec_amaz_albers.tif' --output='copdem_15sec_amaz_albers_breached_d3000' --dist='3000' --min_dist --fill -v --compress_rasters=True

*****************************************
* Welcome to BreachDepressionsLeastCost *
* Powered by WhiteboxTools              *
* www.whiteboxgeo.com                   *
*****************************************
Reading data...
Finding pits: 0%
Finding pits: 1%
Finding pits: 2%
Finding pits: 3%
Finding pits: 4%
Finding pits: 5%
Finding pits: 6%
Finding pits: 7%
Finding pits: 8%
Finding pits: 9%
Finding pits: 10%
Finding pits: 11%
Finding pits: 12%
Finding pits: 13%
Finding pits: 14%
Finding pits: 15%
Finding pits: 16%
Finding pits: 17%
Finding pits: 18%
Finding pits: 19%
Finding pits: 20%
Finding pits: 21%
Finding pits: 22%
Finding pits: 23%
Finding pits: 24%
Finding pits: 25%
Finding pits: 26%
Finding pits: 27%
Finding pits: 28%
Finding pits: 29%
Findi

0

In [7]:
# fill depressions
filled = f'{breach}_filled'
wbt.fill_depressions(
    dem = breach+'.tif', 
    output = filled)

./whitebox_tools --run="FillDepressions" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --dem='copdem_15sec_amaz_albers_breached_d3000.tif' --output='copdem_15sec_amaz_albers_breached_d3000_filled' --fix_flats -v --compress_rasters=True

******************************
* Welcome to FillDepressions *
* Powered by WhiteboxTools   *
* www.whiteboxgeo.com        *
******************************
Reading data...
Finding pit cells: 16%
Finding pit cells: 33%
Finding pit cells: 50%
Finding pit cells: 66%
Finding pit cells: 83%
Finding pit cells: 100%
Filling depressions: 4%
Filling depressions: 9%
Filling depressions: 14%
Filling depressions: 19%
Filling depressions: 23%
Filling depressions: 28%
Filling depressions: 33%
Filling depressions: 38%
Filling depressions: 42%
Filling depressions: 47%
Filling depressions: 52%
Filling depressions: 57%
Filling depressions: 61%
Filling depressions: 66%
Filling depressions: 71%
Filling depressions: 76%
Filling depressions: 80%
Filling depressions: 85%
Filling depr

0

In [11]:
# Perform a flow-accumulation operation. Here I'm using the Qin (2007) multiple flow direction algorithm
# but there are many other options available, including D-infinity.
#
# Stream channels are usually identified as areas of relatively high flow accumulation and are mapped by thresholding
# flow accumulation values. Let's choose a threshold value.
thresh = 25000.0
flowacc = f'{filled}_qinflow_{thresh}'
wbt.qin_flow_accumulation(
    dem = filled+'.tif', 
    output = flowacc+'.tif', 
    out_type="specific contributing area", 
    exponent=10.0, 
    max_slope=45.0, 
    threshold=thresh, 
    log=True, 
    clip=False)


./whitebox_tools --run="QinFlowAccumulation" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --dem='copdem_15sec_amaz_albers_breached_d3000_filled.tif' --output='copdem_15sec_amaz_albers_breached_d3000_filled_qinflow_25000.0_noLog.tif' --out_type=specific contributing area --exponent=10.0 --max_slope=45.0 --threshold='25000.0' -v --compress_rasters=True

**********************************
* Welcome to QinFlowAccumulation *
* Powered by WhiteboxTools       *
* www.whiteboxgeo.com            *
**********************************
Num. inflowing neighbours: 0%
Num. inflowing neighbours: 1%
Num. inflowing neighbours: 2%
Num. inflowing neighbours: 3%
Num. inflowing neighbours: 4%
Num. inflowing neighbours: 5%
Num. inflowing neighbours: 6%
Num. inflowing neighbours: 7%
Num. inflowing neighbours: 8%
Num. inflowing neighbours: 9%
Num. inflowing neighbours: 10%
Num. inflowing neighbours: 11%
Num. inflowing neighbours: 12%
Num. inflowing neighbours: 13%
Num. inflowing neighbours: 14%
Num. inflowing neighbo

0

In [19]:
# Map the streams by thresholding the flow accum raster, using the same convergence threshold used above. This way
# we can be assured that the streams are single-cell wide D8 representation, which is needed for any stream
# network analysis operations.
# streams = flow_accum > math.log(channel_threshold)
flowacc = f'{filled}_qinflow_{thresh}'
streams = f'{flowacc}_streams_t{thresh}'
wbt.greater_than(
    input1 = flowacc+'.tif', 
    input2 = m.log(thresh*5), 
    output = streams+'.tif', 
    incl_equals=False)

./whitebox_tools --run="GreaterThan" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --input1='copdem_15sec_amaz_albers_breached_d3000_filled_qinflow_25000.0.tif' --input2='11.736069016284437' --output='copdem_15sec_amaz_albers_breached_d3000_filled_qinflow_25000.0_streams_t25000.0.tif' -v --compress_rasters=True

****************************
* Welcome to GreaterThan   *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 

0

In [20]:
# Make sure that the outlet is positioned along the stream
shp_foz = 'foz.shp'
shp_snap = 'foz_snap.shp'
wbt.jenson_snap_pour_points(
    pour_pts = shp_foz, 
    streams = streams+'.tif', 
    output = shp_snap, 
    snap_dist = 500)

./whitebox_tools --run="JensonSnapPourPoints" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --pour_pts='foz.shp' --streams='copdem_15sec_amaz_albers_breached_d3000_filled_qinflow_25000.0_streams_t25000.0.tif' --output='foz_snap.shp' --snap_dist='500' -v --compress_rasters=True

***********************************
* Welcome to JensonSnapPourPoints *
* Powered by WhiteboxTools        *
* www.whiteboxgeo.com             *
***********************************
Reading data...
Progress: 0%
Saving data...
Output file written
Elapsed Time (excluding I/O): 0.0s


0

In [21]:
# We need a d8-pointer raster to be able to route flow through the network
d8_pntr = f'{filled}_d8ptr'
wbt.d8_pointer(
    dem = filled+'.tif', 
    output = d8_pntr)

./whitebox_tools --run="D8Pointer" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --dem='copdem_15sec_amaz_albers_breached_d3000_filled.tif' --output='copdem_15sec_amaz_albers_breached_d3000_filled_d8ptr' -v --compress_rasters=True

****************************
* Welcome to D8Pointer     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progress: 42%
Progress: 43%
Progress

0

In [22]:
# calculate watershed
watershed = f'{filled}_watershed'
wbt.watershed(
    d8_pntr = d8_pntr+'.tif', 
    pour_pts = shp_snap, 
    output = watershed)

# Vectorize the watershed polygon for visualization
wbt.raster_to_vector_polygons(
    i = watershed+'.tif', 
    output = watershed+'.shp')

./whitebox_tools --run="Watershed" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --d8_pntr='copdem_15sec_amaz_albers_breached_d3000_filled_d8ptr.tif' --pour_pts='foz_snap.shp' --output='copdem_15sec_amaz_albers_breached_d3000_filled_watershed' -v --compress_rasters=True

****************************
* Welcome to Watershed     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Locating pour points: 0%
Initializing: 1%
Initializing: 2%
Initializing: 3%
Initializing: 4%
Initializing: 5%
Initializing: 6%
Initializing: 7%
Initializing: 8%
Initializing: 9%
Initializing: 10%
Initializing: 11%
Initializing: 12%
Initializing: 13%
Initializing: 14%
Initializing: 15%
Initializing: 16%
Initializing: 17%
Initializing: 18%
Initializing: 19%
Initializing: 20%
Initializing: 21%
Initializing: 22%
Initializing: 23%
Initializing: 24%
Initializing: 25%
Initializing: 26%
Initializing: 27%
Initializing: 28%
Initializing: 29%
Initializing: 30%
Initializing: 31%


0

In [25]:
# basins
sbasins = f'{filled}_sbasins_t{thresh}'
wbt.subbasins(
    d8_pntr = d8_pntr+'.tif',
    streams = streams+'.tif', 
    output = sbasins+'.tif')

# Vectorize the watershed polygon for visualization
# wbt.raster_to_vector_polygons(
#     i = sbasins+'.tif', 
#     output = sbasins+'.shp')


./whitebox_tools --run="Subbasins" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --d8_pntr='copdem_15sec_amaz_albers_breached_d3000_filled_d8ptr.tif' --streams='copdem_15sec_amaz_albers_breached_d3000_filled_qinflow_25000.0_streams_t25000.0.tif' --output='copdem_15sec_amaz_albers_breached_d3000_filled_sbasins_t25000.0.tif' -v --compress_rasters=True

****************************
* Welcome to Subbasins     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Processing streams: 0%
Processing streams: 1%
Processing streams: 2%
Processing streams: 3%
Processing streams: 4%
Processing streams: 5%
Processing streams: 6%
Processing streams: 7%
Processing streams: 8%
Processing streams: 9%
Processing streams: 10%
Processing streams: 11%
Processing streams: 12%
Processing streams: 13%
Processing streams: 14%
Processing streams: 15%
Processing streams: 16%
Processing streams: 17%
Processing streams: 18%
Processing streams: 19%
Processing streams: 20%

0

In [26]:
# trying to derive only large basins
thresh = 2000000000

streams = f'{flowacc}_streams_t{thresh}'
wbt.greater_than(
    input1 = flowacc+'.tif', 
    input2 = m.log(thresh), 
    output = streams+'.tif', 
    incl_equals=False)

# subbasins
sbasins = f'{filled}_sbasins_t{thresh}'
wbt.subbasins(
    d8_pntr = d8_pntr+'.tif',
    streams = streams+'.tif', 
    output = sbasins)

# Vectorize the watershed polygon for visualization
wbt.raster_to_vector_polygons(
    i = sbasins+'.tif', 
    output = sbasins+'.shp')

./whitebox_tools --run="GreaterThan" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --input1='copdem_15sec_amaz_albers_breached_d3000_filled_qinflow_25000.0.tif' --input2='21.416413017506358' --output='copdem_15sec_amaz_albers_breached_d3000_filled_qinflow_25000.0_streams_t2000000000.tif' -v --compress_rasters=True

****************************
* Welcome to GreaterThan   *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progres

0

In [27]:
# longest flow path
longest = f'{filled}_longestflowpath'
wbt.longest_flowpath(
    dem = filled+'.tif', 
    basins = watershed+'.tif', 
    output =  longest)

./whitebox_tools --run="LongestFlowpath" --wd="/mnt/d/dados_SPAMLab/amaz_hidro" --dem='copdem_15sec_amaz_albers_breached_d3000_filled.tif' --basins='copdem_15sec_amaz_albers_breached_d3000_filled_watershed.tif' --output='copdem_15sec_amaz_albers_breached_d3000_filled_longestflowpath' -v --compress_rasters=True

******************************
* Welcome to LongestFlowpath *
* Powered by WhiteboxTools   *
* www.whiteboxgeo.com        *
******************************
Reading data...
Flow directions: 0%
Flow directions: 1%
Flow directions: 2%
Flow directions: 3%
Flow directions: 4%
Flow directions: 5%
Flow directions: 6%
Flow directions: 7%
Flow directions: 8%
Flow directions: 9%
Flow directions: 10%
Flow directions: 11%
Flow directions: 12%
Flow directions: 13%
Flow directions: 14%
Flow directions: 15%
Flow directions: 16%
Flow directions: 17%
Flow directions: 18%
Flow directions: 19%
Flow directions: 20%
Flow directions: 21%
Flow directions: 22%
Flow directions: 23%
Flow directions: 24%
F

0